In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class FPN(nn.Module):
    r"""Feature Pyramid Network.

    이 코드는 'Feature Pyramid Networks for Object
    Detection'라는 논문구현된 코드입니다
    "https://arxiv.org/abs/1612.03144" 참고바랍니다


    Args:
        in_channels (List[int]): input feature map들의 channels.
        out_channels (int): output channel
        extra_level (bool): Number of output scales.
            Default: `True
        upsample_cfg (dict): Config dict for interpolate layer.
            Default: `dict(mode='nearest')`


    1) top-down 전 channel을 맞춰주기 위하여 convolution layer 통과
    2) top-down을 통해 서로 다른 level의 feature map 교환
    3) extra level의 feature map을 추가로 구성 (선택 사항)
    """

    def __init__(
        self,
        in_channels,
        out_channels,
        extra_level=True,
        upsample_cfg=dict(mode="nearest"),
    ):
        super(FPN, self).__init__()
        assert isinstance(in_channels, list)
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_ins = len(in_channels)
        self.upsample_cfg = upsample_cfg.copy()
        self.backbone_end_level = self.num_ins
        self.extra_level = extra_level

        self.lateral_convs = nn.ModuleList()
        self.fpn_convs = nn.ModuleList()

        """
        input list의 길이만큼 fpn_conv, lateral_conv 생성
        fpn_conv: top-down 수행 전 channel을 맞춰주는 convolution
        lateral_conv: top-down 수행 후 학습을 위해 통과하는 convolution
        """
        for i in range(self.backbone_end_level):
            l_conv = nn.Conv2d(
                in_channels[i], out_channels, kernel_size=1, stride=1, padding=0
            )
            fpn_conv = nn.Conv2d(
                out_channels, out_channels, kernel_size=3, stride=1, padding=1
            )

            self.lateral_convs.append(l_conv)
            self.fpn_convs.append(fpn_conv)

        self.normal_init(self.fpn_convs, 0, 0.01)
        self.normal_init(self.lateral_convs, 0, 0.01)

        if self.extra_level:
            in_channels = self.in_channels[self.backbone_end_level - 1]
            self.extra_conv = nn.Conv2d(
                out_channels, out_channels, kernel_size=3, stride=2, padding=1
            )

            self.normal_init(self.extra_conv, 0, 0.01)

    # default init_weights for conv(msra) and norm in ConvModule
    def normal_init(self, convs, mean, stddev, truncated=False):
        """
        weight initialization
        """
        if isinstance(convs, nn.ModuleList):
            for conv in convs:
                conv.weight.data.normal_(mean, stddev)
                conv.bias.data.zero_()
        else:
            convs.weight.data.normal_(mean, stddev)
            convs.bias.data.zero_()

    """
        inputs: list of feature maps from backbone
        outs: list of feature maps
                FPN을 통과한 feature map, input 과 shape 동일
                self.extra_level인 True인 경우 feature map 하나 추가
    """

    def forward(self, inputs):
        """Forward function."""
        assert len(inputs) == len(self.in_channels)

        # build laterals
        # use self.lateral_convs
        laterals = [
            lateral_conv(inputs[i]) for i, lateral_conv in enumerate(self.lateral_convs)
        ]

        # build top-down path
        # use F.interpolate(laterals[i], size, **self.upsample_cfg)

        """코드추가"""

        # build outputs
        # use self.fpn_convs
        # part 1: from original levels
        outs = ["""코드추가"""]

        # part 2: add extra levels
        # use self.extra_level
        if self.extra_level:
            outs.append("""코드추가""")

        return tuple(outs)

In [ ]:
import torch


"""
backbone에서 나온 feature map dummy
"""
in_channels = [2, 3, 5, 7]
scales = [340, 170, 84, 43]
inputs = [torch.rand(1, c, s, s) for c, s in zip(in_channels, scales)]

"""
init FPN
"""
self = FPN(in_channels, 11, extra_level=False).eval()
outputs = self.forward(inputs)

"""
outputs[0].shape = torch.Size([1, 11, 340, 340])
outputs[1].shape = torch.Size([1, 11, 170, 170])
outputs[2].shape = torch.Size([1, 11, 84, 84])
outputs[3].shape = torch.Size([1, 11, 43, 43])
위의 결과가 나오면 성공
"""
"""
FPN을 통과하여 서로 다른 level의 feature를 교환함으로써 feature의 semantic을 강화
"""
for i in range(len(outputs)):
    print(f"outputs[{i}].shape = {outputs[i].shape}")

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.
